# Overview #
In this notebook, you leverage the cpdctl command to copy assets from one deployment space of Cloud Pak for Data to another deployment space. This is typically applicable in MLOps process where you want to promote the relevant data assets and AI models from an initial deployment space, such as Pre-Prod (or UAT) deployment space, to another deployment space, such as Prod deployment space.

For purposes of AI governance as well as CI/CD component of MLOps, it is important to be able to automate the process of promoting relevant assets from one deployment space owner by development team to another deployment space owned by the QA team.

Please note that the two deployment spaces could exist on the same Cloud Pak for Data cluster or could belong to two different Cloud Pak for Data clusters in different environments.

### Execution Steps ###
In order to leverage cpdctl to copy assets from one deployment space to another, you need to provide the following information:
- SOURCE_CPD_URL : The url for the source Cloud Pak for Data cluster
- SOURCE_CPD_USERNAME: The username for the source Cloud Pak for Data cluster
- SOURCE_CPD_PASSWORD: The password for the source Cloud Pak for Data cluster
- TARGET_CPD_URL : The url for the target Cloud Pak for Data cluster
- TARGET_CPD_USERNAME: The username for the target Cloud Pak for Data cluster
- TARGET_CPD_PASSWORD: The password for the target Cloud Pak for Data cluster
- SOURCE_DEPLOYMENT_SPACE_NAME: The name of the deployment space on the source Cloud Pak for Data cluster (source deployment space should exist).
- TARGET_DEPLOYMENT_SPACE_NAME: The name of the deployment space on the target Cloud Pak for Data cluster (any name you choose since it will be created).
- TARGET_MODEL_NAME: The name of the model in the atarget deployment space (any name you choose since it will be created).

Given this information, this notebook will define the cpdctl contexts corresponding to the source and target Cloud Pak for Data clusters, create a new deployment space on the target cluster, and copy all assets from the source deployment space to the target deployment space.

The code assumes that the SOURCE_DEPLOYMENT_SPACE_NAME exists and contains the relevant assets and that the TARGET_DEPLOYMENT_SPACE_NAME also exists but has no assets.

In [1]:
# Import required libraries and modules
import base64
import json
import os
import platform
import requests
import tarfile
import zipfile
import jmespath
import subprocess
from IPython.display import display, HTML

## Credentials for Cloud Pak for Data and Deployment Spaces 
Please update the information below to include the Deployment Spaces and credentials for Cloud Pak for Data. The provided credentials should have the permissions to edit and update deployment spaces. Typically, **datascientist** user with Data Scientist role can have the right permissions to manage deployment spaces.

For the Cloud Pak for Data url, if you are running on an environment hosted on Skytap, make sure to use the internal URL. You can get the correct url by running the following command in the openshift cluster.
**oc get ZenService lite-cr -o jsonpath="{.status.url}{'\n'}"**

Alternatively, you can log into the OpenShift web consolve, navigate to Networking ==> Routes and find the Cloud Pak for Data url under Location.

In [2]:
SOURCE_CPD_USERNAME = '<USERNAME_WITH_ACCESS_TO_SOURCE_DEPLOYMENT_SPACE>' # for example: datascientist
SOURCE_CPD_PASSWORD = '<PASSWORD_FOR_SOURCE_CPD_USERNAME>'
SOURCE_CPD_URL = '<CLOUD PAK FOR DATA URL FOR SOURCE CLUSTER>' #typically, this would be https://cpd-cpd-instance.apps.demo.ibmdte.net

TARGET_CPD_USERNAME = '<USERNAME_WITH_ACCESS_TO_TARGET_DEPLOYMENT_SPACE>' # for example: datascientist
TARGET_CPD_PASSWORD = '<PASSWORD_FOR_TARGET_CPD_USERNAME>'
TARGET_CPD_URL = '<CLOUD PAK FOR DATA URL FOR TARGET CLUSTER>' #typically, this would be https://cpd-cpd-instance.apps.demo.ibmdte.net

# Code assumes both deployment spaces exist
SOURCE_DEPLOYMENT_SPACE_NAME='<NAME OF SOURCE DEPLOYMENT SPACE>' # For example: 'churnUATspace'
TARGET_DEPLOYMENT_SPACE_NAME='<NAME OF TARGET DEPLOYMENT SPACE>' # For example: 'churn_prod_space'

TARGET_MODEL_NAME='<TARGET DEPLOYMENT NAME>' # Any name to assign to target deployment, for example; ChurnPredictionProd

### Install the version v1.1.229 of `cpdctl`


In [3]:
PLATFORM = platform.system().lower()
CPDCTL_ARCH = "{}_amd64".format(PLATFORM)
CPDCTL_RELEASES_URL="https://api.github.com/repos/IBM/cpdctl/releases"
CWD = os.getcwd()
PATH = os.environ['PATH']
CPDCONFIG = os.path.join(CWD, '.cpdctl.config.yml')
version='v1.4.116'

response = requests.get(CPDCTL_RELEASES_URL)
asset_version = next(a for a in response.json() if version==a['tag_name'])
#assets = response.json()[0]['assets']
assets=asset_version['assets']
platform_asset = next(a for a in assets if CPDCTL_ARCH in a['name'])
cpdctl_url = platform_asset['url']
cpdctl_file_name = platform_asset['name']

response = requests.get(cpdctl_url, headers={'Accept': 'application/octet-stream'})
with open(cpdctl_file_name, 'wb') as f:
    f.write(response.content)
    
display(HTML('<code>cpdctl</code> binary downloaded from: <a href="{}">{}</a>'.format(platform_asset['browser_download_url'], platform_asset['name'])))

In [4]:
%%capture

%env PATH={CWD}:{PATH}
%env CPDCONFIG={CPDCONFIG}

In [5]:
if cpdctl_file_name.endswith('tar.gz'):
    with tarfile.open(cpdctl_file_name, "r:gz") as tar:
        tar.extractall()
elif cpdctl_file_name.endswith('zip'):
    with zipfile.ZipFile(cpdctl_file_name, 'r') as zf:
        zf.extractall()

if CPDCONFIG and os.path.exists(CPDCONFIG):
    os.remove(CPDCONFIG)
    
version_r = ! cpdctl version
CPDCTL_VERSION = version_r.s

print("cpdctl version: {}".format(CPDCTL_VERSION))

cpdctl version: 1.1.229


In [6]:
!which cpdctl

/home/wsuser/work/cpdctl


# CPDCTL Demo

AI Lifecycle automation using `cpdctl` CLI tool with one CPD 4.0 cluster and two different deployment spaces
- 'UAT': UAT or Pre-Prod Deployment Space
- 'PROD': Prod Deployment Space

### List Project and Spaces in CPD 4.5

List Project and Spaces in CPD 4.5 to make sure about Zero Configuration in cpdctl v1.1.229: https://github.com/IBM/cpdctl#zero-configuration

In [7]:
! cpdctl space list

...
ID                                     Name               Created                    Description   State    Tags   
e1f2047d-16e0-447e-a854-464f7edb5346   dev                2022-08-01T14:55:36.108Z                 active   []   
c7faaed9-43bf-4444-b548-98c15fac8c14   churnUATspace      2022-08-01T15:12:07.173Z                 active   []   
57e67b8e-97cb-401f-a4d2-c9234100de04   churn_prod_space   2022-08-02T09:37:55.724Z                 active   []   
bb4bd7b5-ae2b-4e7d-abe6-e455e8d466d9   churn-dev          2022-08-02T11:32:47.588Z                 active   []   


List available projects in "source" context

In [8]:
! cpdctl project list

...
ID                                     Name                        Created                    Description   Tags   
39243527-db24-4905-9d3d-fed781f85e76   Customer Churn Prediction   2022-08-01T13:56:20.624Z                 []   
f11aa3ea-184b-46fb-9da6-7e2ec4fcbc2b   mlopsdev                    2022-08-02T11:00:18.602Z                 []   


## Get Deployment Space IDs

In [29]:
def getSpaceID(name):
    cmd = "cpdctl space list --output json -j \"(resources[?entity.name == " + "'" + name + "'" + "].metadata.id)[0]\" --raw-output"
    
    print("executing command: ", cmd)
    
    space_id = subprocess.getoutput(cmd)
    
    return space_id


In [30]:
source_deployment_spaceID=getSpaceID(SOURCE_DEPLOYMENT_SPACE_NAME)
print("Source Deployment Space ID: ", source_deployment_spaceID)

executing command:  cpdctl space list --output json -j "(resources[?entity.name == 'churnUATspace'].metadata.id)[0]" --raw-output
Source Deployment Space ID:  c7faaed9-43bf-4444-b548-98c15fac8c14


Export All Assets from the dev deploymentspace

In [32]:
EXPORT = {
    'all_assets': True
}
EXPORT_JSON = json.dumps(EXPORT)

result = ! cpdctl asset export start --space-id {source_deployment_spaceID} --assets '{EXPORT_JSON}' --name source-space-assets --output json --jmes-query "metadata.id"
EXPORT_ID = result.s
print("The new export with ID: {}".format(EXPORT_ID))

The new export with ID: "29e21c7e-9b6d-4ba6-a638-33b01235a31c"


In [33]:
! cpdctl asset export download --space-id {source_deployment_spaceID} --export-id {EXPORT_ID} --output-file source-space-assets.zip

...
OK
Output written to source-space-assets.zip


In [34]:
TARGET_SPACE_ID=getSpaceID(TARGET_DEPLOYMENT_SPACE_NAME)
print("Target Deployment Space ID: ", TARGET_SPACE_ID)

executing command:  cpdctl space list --output json -j "(resources[?entity.name == 'churn_prod_space'].metadata.id)[0]" --raw-output
Target Deployment Space ID:  57e67b8e-97cb-401f-a4d2-c9234100de04


In [37]:
result = ! cpdctl asset import start --space-id {TARGET_SPACE_ID} --import-file source-space-assets.zip --output json --jmes-query "metadata.id" --raw-output
IMPORT_ID = result.s
print("The new import ID is: {}".format(IMPORT_ID))

The new import ID is: 7aaf2fb4-f33d-4e13-8fbc-85188b84da92


In [38]:
! cpdctl asset import get --space-id {TARGET_SPACE_ID} --import-id {IMPORT_ID}

...
              
ID:        7aaf2fb4-f33d-4e13-8fbc-85188b84da92   
Created:   2022-08-03T07:43:35.860Z   
State:     completed   


In [39]:
! cpdctl ml model list --space-id {TARGET_SPACE_ID}

...
ID                                     Name          Created                    Type        Tags   
3534d2f3-0f85-45a0-89cc-7bc2e729a2f3   Churn Model   2022-08-03T07:43:39.002Z   mllib_3.0   []   


In [40]:
result = ! cpdctl ml model list --space-id {TARGET_SPACE_ID} --output json --jmes-query "resources[0].metadata.id" --raw-output
TARGET_MODEL_ID = result.s
print("TARGET model ID is: {}".format(TARGET_MODEL_ID))

TARGET model ID is: 3534d2f3-0f85-45a0-89cc-7bc2e729a2f3


In [41]:
ASSET_JSON = json.dumps({"id": TARGET_MODEL_ID})
ONLINE_JSON = json.dumps({})

! cpdctl ml deployment create --space-id {TARGET_SPACE_ID} --asset '{ASSET_JSON}' --online '{ONLINE_JSON}' --name {TARGET_MODEL_NAME}

...
              
ID:        b7f55968-04be-4b93-ab31-7c243867694e   
Name:      ChurnPredictionProd   
Created:   2022-08-03T07:44:09.975Z   
State:     ready   
Tags:      []   


## Verify Deployment ##
At this point, a new model deployment should appear in your target deployment space.

1. Navigate to your target deployment space: Select the Navigation Menu (top left hamburger icon), right click on *Deployments*, and select *Open Link in New Tab*.
2. In the new tab, select the *Spaces* tab and click on the name of the target deployment space (for example, *qa*)
3. On the Deployments/\<target space name\> page, click on *Deployments* tab.
4. Verify the *TARGET_MODEL_NAME* appears in the list of deployed models. Click the *TARGET_MODEL_NAME* model.
5. On the Deployed model page, click the Test tab and provide a sample test to validate the model returns predictions as expected.

    

In [42]:
cpdtoken=os.environ['USER_ACCESS_TOKEN']
wml_credentials = {
"token": cpdtoken,
"instance_id" : "openshift",
"url": os.environ['RUNTIME_ENV_APSX_URL'],
"version": "4.8"
}

from ibm_watson_machine_learning import APIClient
client = APIClient(wml_credentials)


In [43]:
def getSpaceIDwml(wml_client,space_name):
    spaces = wml_client.spaces.get_details()['resources'];
    spaceList = next(item for item in spaces if item['entity']['name']==space_name)
    spaceID = spaceList['metadata']['id']
    return spaceID

In [44]:
space_name=TARGET_DEPLOYMENT_SPACE_NAME
space_id = getSpaceIDwml(client,space_name)
print(space_id)
client.set.default_space(space_id)

57e67b8e-97cb-401f-a4d2-c9234100de04


'SUCCESS'

In [45]:
space_details=client.spaces.get_details(space_id)

In [46]:
client.repository.list_models()

------------------------------------  -----------  ------------------------  ---------
ID                                    NAME         CREATED                   TYPE
3534d2f3-0f85-45a0-89cc-7bc2e729a2f3  Churn Model  2022-08-03T07:43:39.002Z  mllib_3.0
------------------------------------  -----------  ------------------------  ---------


In [47]:
def getModelDetails(wml_client,deployment_name):
    models = wml_client.deployments.get_details()['resources'];
    modelList = next(item for item in models if item['entity']['name']==deployment_name)
    #modelID = modelList['metadata']['id']
    #return modelID
    return modelList
    

In [48]:
model_name=TARGET_MODEL_NAME
model_details = getModelDetails(client,model_name)
print(model_details)

{'entity': {'asset': {'id': '3534d2f3-0f85-45a0-89cc-7bc2e729a2f3'}, 'custom': {}, 'deployed_asset_type': 'model', 'hardware_spec': {'id': 'e7ed1d6c-2e89-42d7-aed5-863b972c1d2b', 'name': 'S', 'num_nodes': 1}, 'name': 'ChurnPredictionProd', 'online': {}, 'space_id': '57e67b8e-97cb-401f-a4d2-c9234100de04', 'status': {'online_url': {'url': 'https://internal-nginx-svc.cpd-instance.svc.cluster.local:12443/ml/v4/deployments/b7f55968-04be-4b93-ab31-7c243867694e/predictions'}, 'serving_urls': ['https://internal-nginx-svc.cpd-instance.svc.cluster.local:12443/ml/v4/deployments/b7f55968-04be-4b93-ab31-7c243867694e/predictions'], 'state': 'ready'}}, 'metadata': {'created_at': '2022-08-03T07:44:09.975Z', 'id': 'b7f55968-04be-4b93-ab31-7c243867694e', 'modified_at': '2022-08-03T07:44:09.975Z', 'name': 'ChurnPredictionProd', 'owner': '1000331014', 'space_id': '57e67b8e-97cb-401f-a4d2-c9234100de04'}, 'system': {'warnings': [{'id': 'Deprecated', 'message': 'Software specification spark-mllib_3.0 is depr

In [49]:
# Score the model on a test dataset
scoring_payload = {
    "input_data": [{
        'fields': ['ID', 'LONGDISTANCE', 'INTERNATIONAL', 'LOCAL', 'DROPPED', 'PAYMETHOD', 'LOCALBILLTYPE', 'LONGDISTANCEBILLTYPE', 'USAGE', 'RATEPLAN', 'GENDER','STATUS', 'CHILDREN', 'ESTINCOME', 'CAROWNER', 'AGE'],
        'values': [[1,28,0,60,0,"Auto","FreeLocal","Standard",89,4,"F","M",1,23000,"N",45]]}]
}



In [50]:
modelID=model_details['metadata']['id']
predictions = client.deployments.score(modelID, scoring_payload)
print(json.dumps(predictions, indent=2))

{
  "predictions": [
    {
      "fields": [
        "ID",
        "LONGDISTANCE",
        "INTERNATIONAL",
        "LOCAL",
        "DROPPED",
        "PAYMETHOD",
        "LOCALBILLTYPE",
        "LONGDISTANCEBILLTYPE",
        "USAGE",
        "RATEPLAN",
        "GENDER",
        "STATUS",
        "CHILDREN",
        "ESTINCOME",
        "CAROWNER",
        "AGE",
        "PAYMETHOD_IX",
        "LOCALBILLTYPE_IX",
        "LONGDISTANCEBILLTYPE_IX",
        "GENDER_IX",
        "STATUS_IX",
        "CAROWNER_IX",
        "label",
        "features",
        "rawPrediction",
        "probability",
        "prediction",
        "predictedLabel"
      ],
      "values": [
        [
          1,
          28,
          0,
          60,
          0,
          "Auto",
          "FreeLocal",
          "Standard",
          89,
          4,
          "F",
          "M",
          1,
          23000.0,
          "N",
          45.0,
          1.0,
          1.0,
          0.0,
          0.0

# Summary #
This notebook illustrates one approach to apply CI/CD against your models where you can automate continuous integration and delivery of models from UAT (or preProd) deployment space to production deployment space.